# Data Engineering 1: Graded Lab 01
---------------

#### Grading
For this graded lab you can get a total of 10 points. These 10 points count 10% of your final grade for the course.

#### Start
Start of the Graded Lab 01 is **Thursday, March 21st at 23:55**.

#### Deadline
Deadline for the submission of the Graded Lab 01 is **Thursday, April 11th at 23:59**.

#### Note
Check each result carefully. Use data filter, cleaning, and transformation methods wherever needed. The data can sometimes be really messy and have hidden issues.

#### Submission
You are allowed to submit the solution in groups of **two or three** students.
Submit your GradedLab01.ipynb file renamed to FirstnameStudent01LastnameStudent01_FirstnameStudent02LastnameStudent02_FirstnameStudent03LastnameStudent03.ipynb in moodle.   
Please submit a runnable python jupyter notebook file.
All other submissions will be rejected and graded with 0 points.

##### Task 01: Structured Data [4 points].    
The imdb.csv file contains a dataset extracted from IMDB with several attributes. For example, the title, plot, and the language of the movies. Read the imdb.csv file in a pandas dataframe and try to answer the following questions.

##### __(a)  What is the min, max, and the average value of the attribute 'rating' for all movies? [0.5 points].__ 

In [1]:
import pandas as pd

imdb = pd.read_csv('imdb.csv')

print(f"Lowest rating: {imdb['rating'][imdb['kind'] == 'movie'].min()}")
print(f"Highest rating: {imdb['rating'][imdb['kind'] == 'movie'].max()}")
print(f"Average rating: {imdb['rating'][imdb['kind'] == 'movie'].mean():.2f}")

Lowest rating: 8.1
Highest rating: 9.3
Average rating: 8.35


##### __(b) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' for all movies? [0.5 points].__ 

In [2]:
# Data cleansing
imdb['cumulative worldwide gross'] = imdb['cumulative worldwide gross'].str.replace(r'[\$,]', '', regex=True)
imdb['cumulative worldwide gross'] = imdb['cumulative worldwide gross'].str.split().str[0]
imdb['cumulative worldwide gross'] = imdb['cumulative worldwide gross'].str.replace('000000000', '0', regex=True)
imdb['cumulative worldwide gross'] = pd.to_numeric(imdb['cumulative worldwide gross'], errors='coerce')

In [3]:

# Queries
print(f"Lowest cumulative worldwide gross: {imdb['cumulative worldwide gross'][imdb['kind'] == 'movie'].min()}")
print(f"Highest cumulative worldwide gross: {imdb['cumulative worldwide gross'][imdb['kind'] == 'movie'].max()}")
print(f"Average cumulative worldwide gross: {imdb['cumulative worldwide gross'][imdb['kind'] == 'movie'].mean():.2f}")

Lowest cumulative worldwide gross: 6540000
Highest cumulative worldwide gross: 2797800564
Average cumulative worldwide gross: 390489624.68


##### __(c) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' grouped by genre of the movies? [1 point].__ 

In [4]:
# Convert values of this column to lists
imdb['genres'] = imdb['genres'].apply(eval)

In [5]:
# Dataframe with only movies
df_movies = imdb[imdb['kind'] == 'movie']

# Get all unique genres
genres = []
for cell in df_movies['genres']:
    for element in cell:
        genres.append(element)

genres = list(set(genres))
genres.sort()
df_worldwide = pd.DataFrame(columns=['genre', 'min', 'max', 'mean'])

for genre in genres:
    minima = df_movies[df_movies['genres'].apply(lambda x: genre in x)]["cumulative worldwide gross"].min()
    maxima = df_movies[df_movies['genres'].apply(lambda x: genre in x)]["cumulative worldwide gross"].max()
    avg = df_movies[df_movies['genres'].apply(lambda x: genre in x)]["cumulative worldwide gross"].mean()
    new_row = pd.DataFrame({'genre': [genre], 'min': [minima], 'max': [maxima], 'mean': [avg]})
    df_worldwide = pd.concat([df_worldwide, new_row], ignore_index=True, axis=0)
    
df_worldwide


C:\Users\jonas\AppData\Local\Temp\ipykernel_22124\2181792769.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_worldwide = pd.concat([df_worldwide, new_row], ignore_index=True, axis=0)


,genre,min,max,mean
0,Action,24215385,2797800564,6.331150e+08
1,Adventure,56255142,2797800564,6.941635e+08
2,Animation,373554033,1068879522,6.597255e+08
3,Biography,56255142,392000694,2.393391e+08
4,Comedy,46189568,1068879522,5.031645e+08
5,Crime,6540000,1084939099,2.846189e+08
6,Drama,7390108,2797800564,3.398912e+08
7,Family,373554033,1068879522,6.597255e+08
8,Fantasy,165800000,1068879522,5.482189e+08
9,History,187733202,400176459,2.548299e+08


##### __(d) What is the profit (defined as cumulative worldwide gross - budget) of each movie? [1 point].__ 

In [6]:
# Data cleansing
imdb['budget'] = imdb['budget'].str.replace(r'[\$,]', '', regex=True)
imdb['budget'] = imdb['budget'].str.split().str[0]
imdb['budget'] = imdb['budget'].str.replace('000000000', '0', regex=True)
imdb['budget'] = pd.to_numeric(imdb['budget'], errors='coerce')

In [7]:
# Queries
imdb['net profit'] = imdb['cumulative worldwide gross'] - imdb['budget']
imdb[['title', 'net profit']][imdb['kind'] == 'movie']

,title,net profit
0,The Shawshank Redemption,33500000
1,The Godfather,239066411
2,The Dark Knight,819558444
3,Schindler's List,199000000
4,Pulp Fiction,205928762
...,...,...
92,Pirates of the Caribbean: The Curse of the Bla...,514264015
93,Aladdin,476050219
94,Beauty and the Beast,399967620
95,The Help,191639112


##### __(e)  What is the min, max, and the average value of the atttribute 'opening weekend united states' per month? [1 point].__ 

In [8]:
#  Assumption: the year does not matter
#  Split 'opening weekend united states' into two columns
import re
split_data = imdb['opening weekend united states'].str.split(' ', expand=True)

# Create a new DataFrame with 'value' and 'date' columns
# Create a new DataFrame with 'value' and 'date' columns
new_data = pd.DataFrame()
new_data['value'] = split_data[0]
new_data['value']= [int(re.sub(r'[^\d]', '', gross)) for gross in new_data['value']]
new_data['value'] = new_data[new_data['value'] != 0]
new_data['dateday'] = split_data[1]
new_data['datemonth'] = split_data[2]
new_data = new_data[new_data['datemonth'] != 'NaN']
new_data['dateyear'] = split_data[3]
new_data.head(97)

data2 = {'value': new_data['value'], 'month': new_data['datemonth']}

min = pd.DataFrame(data2).groupby('month').min()
max = pd.DataFrame(data2).groupby('month').max()
avg = pd.DataFrame(data2).groupby('month').mean()




# Merge and name the columns after the previous DataFrame
revenue_month = pd.merge(min, max, on='month', how='outer', suffixes=('_min', '_max'))
revenue_month = pd.merge(revenue_month, avg, on='month', how='outer')
revenue_month = revenue_month.rename(columns={'value': 'revenue_avg'})
revenue_month

,value_min,value_max,revenue_avg
month,,,
Apr,12562.0,357115007.0,1.102430e+08
Aug,645363.0,26681262.0,1.709731e+07
Dec,179953.0,35363376.0,1.287903e+07
Feb,13766814.0,41062440.0,3.024530e+07
Jan,15172.0,17754905.0,7.363771e+06
Jul,137301.0,160887295.0,4.199736e+07
Jun,340456.0,110307189.0,3.929574e+07
Mar,235488.0,88411916.0,2.360451e+07
May,34819017.0,68108790.0,5.146390e+07


#### Task 02: Unstructured Data [6 points]. 
This time use the files imdb.csv from Task 01 and the imdb2.csv file. In this task we mainly use the content of the attribute 'plot' for the implementation of a retrieval system.

##### __(a)  Read the files imdb.csv and imdb2.csv in a dataframe called imdb. Add a column which is called 'plotterms' to each item. The new column should contain all terms (lower-case and cleaned from special characters) from the plot attribute which are not contained in the stopwords_english.txt file. [0.5 points]__ 

In [9]:
import pandas as pd


imdb2 = pd.read_csv('imdb2.csv')
imdb = pd.read_csv('imdb.csv')
imdb = pd.concat([imdb2, imdb], ignore_index=True)

#Creating Stopword Pattern for later replacement/ delete of said words
stopwords = pd.read_csv('stopwords_english.txt',header=None,names=['stopwords'])
stopwords = stopwords["stopwords"].tolist()
stopwords_pattern = r'\b(?:{})\b'.format('|'.join(stopwords))


# Pattern for replacement of special chars
pattern = r'[^\w\s]'
#Replace special chars with ""
imdb['plotterms'] = imdb['plot'].str.replace(pattern,"",regex=True).str.lower()
#replace stopwords with ""
imdb['plotterms'] = imdb['plotterms'].str.replace(stopwords_pattern,"",regex=True)
#clear out excessive whitespaces
pattern_whitespaces = r"\s+"
imdb['plotterms'] = imdb['plotterms'].str.replace(pattern_whitespaces," ",regex=True)
imdb



,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms
0,120669,movie,Fear and Loathing in Las Vegas,['English'],"['Adventure', 'Comedy', 'Drama']",7.5,Terry Gilliam,0,0,0,['An oddball journalist and his psychopathic l...,oddball journalist psychopathic lawyer travel...
1,1343092,movie,The Great Gatsby,['English'],"['Drama', 'Romance']",7.2,Baz Luhrmann,0,0,0,"['A writer and wall street trader, Nick, finds...",writer wall street trader nick finds drawn li...
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,['Dave has the American Dream and hates it. Hi...,dave american dream hates eccentric friends ha...
3,2267368,movie,Joshua Tree,['English'],['Drama'],6.6,Li Cheng,0,0,0,['Set in the U.S. heartland following the 2008...,set heartland 2008 economic collapse divorced ...
4,13560124,movie,After America,"['English', 'Somali']","['Drama', 'Fantasy', 'Mystery']",4.5,Jake Yuzna,0,0,0,['A group of criminal justice de-escalation wo...,group criminal justice deescalation workers m...
...,...,...,...,...,...,...,...,...,...,...,...,...
162,2085059,tv series,Black Mirror,['English'],"['Drama', 'Mystery', 'Sci-Fi', 'Thriller']",8.8,NaN,000000000,000000000,000000000,"[""An anthology series exploring a twisted, hig...",anthology series exploring twisted hightech m...
163,2707408,tv series,Narcos,"['English', 'Spanish']","['Biography', 'Crime', 'Drama', 'Thriller']",8.8,NaN,000000000,000000000,000000000,['A chronicled look at the criminal exploits o...,chronicled criminal exploits colombian drug l...
164,98936,tv series,Twin Peaks,"['English', 'Icelandic', 'Afrikaans', 'Norwegi...","['Crime', 'Drama', 'Mystery', 'Thriller']",8.8,NaN,000000000,000000000,000000000,['An idiosyncratic FBI agent investigates the ...,idiosyncratic fbi agent investigates murder y...
165,4934214,tv series,Taskmaster,['English'],"['Comedy', 'Game-Show']",9.0,NaN,000000000,000000000,000000000,['Five comedians are set tasks challenging the...,comedians set tasks challenging creativity wi...


##### __(b)  Create an inverted index for the terms in the 'plotterms' column. Use a datastructure of your choice for the implementation [0.5 points]__ 

In [10]:
#Creating Inverted Index Dataframe
inverted_index = pd.DataFrame({
    "Term": [],
    "Movie_IDs": []
})

for index, row in imdb.iterrows(): # Itterate through each row of the dataframe 
    movie_id = row["id"] # Get Movie ID from Every Movie
    for term in imdb.loc[index]["plotterms"].split(): # Itterate through every word in plotterms of said row
        if (inverted_index["Term"] != term).all(): # If plot word not already indexed
            new_row = pd.DataFrame({"Term": [term], "Movie_IDs": [[movie_id]]}) # generate new row for inverted index
            inverted_index = pd.concat([inverted_index, new_row], ignore_index=True) # add plotterm and movieid to inverted index
        elif movie_id not in inverted_index.loc[inverted_index["Term"] == term, "Movie_IDs"].iloc[0]: # if plotterm already indexed but not indexed to this movie
            inverted_index.loc[inverted_index["Term"] == term, "Movie_IDs"].iloc[0].append(movie_id) # add movieid to plotterm index



inverted_index



,Term,Movie_IDs
0,oddball,[120669]
1,journalist,"[120669, 12392504]"
2,psychopathic,"[120669, 477348]"
3,lawyer,"[120669, 2802850, 3032476]"
4,travel,"[120669, 816692, 6966692, 266543, 2395695, 286..."
...,...,...
5552,principles,[200276]
5553,tactical,[200276]
5554,insights,[200276]
5555,aspects,[200276]


##### __(c) Take the inverted index and create the posting list for the query 'american'. Return the posting list as well as the top5 items scored by the rating attribute. [0.5 points]__ 

In [11]:
# get Movie_IDs for the word american
term_american = inverted_index[inverted_index["Term"]== "american"]
movie_id_american = term_american["Movie_IDs"].iloc[0]

#Print postinglist for Movies with the plotterms "american" 
print(f"Posting list dream & american : {movie_id_american}")

# Query for all films with american in plotterms and sort it by best rating
query_american = imdb[imdb["id"].isin(movie_id_american)].sort_values(by="rating",ascending=False)
query_american.head(5) # Show only top 5 results


Posting list dream & american : [120669, 1343092, 2069861, 2267368, 13560124, 5092484, 2365001, 373862, 88763, 46912, 112471, 381681, 306414, 1877514, 98769]


,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms
110,306414,tv series,The Wire,"['English', 'Greek', 'Mandarin', 'Spanish']","['Crime', 'Drama', 'Thriller']",9.3,NaN,000000000,000000000,000000000,"['The Baltimore drug scene, as seen through th...",baltimore drug scene eyes drug dealers law en...
124,1877514,tv mini series,The Vietnam War,['English'],"['Documentary', 'History', 'War']",9.1,NaN,000000000,000000000,000000000,"[""A comprehensive history of the United States...",comprehensive history united involvement bitt...
143,98769,tv mini series,The Civil War,['English'],"['Documentary', 'History', 'War']",9.0,NaN,000000000,000000000,000000000,['A comprehensive survey of the American Civil...,comprehensive survey american civil war highl...
26,88763,movie,Back to the Future,['English'],"['Adventure', 'Comedy', 'Sci-Fi']",8.6,Robert Zemeckis,"$381,109,762","$11,332,134, 07 Jul 1985","$19,000,000 (estimated)","Marty McFly, a typical American teenager of th...",marty mcfly typical american teenager eighties...
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,['Dave has the American Dream and hates it. Hi...,dave american dream hates eccentric friends ha...


##### __(d) Take the inverted index and create the posting lists for the query terms 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot). Your merging algorithm should be efficient and reduce the number of comparison to a minimum. Don't use existing python methods (like intersect or in) for the merge algorithm. Return the merged posting list as well as the top5 items scored by the rating attribute. [1 point]__ 

In [12]:
# American Movie_IDs
movie_id_american
# get Movie_IDs for term dream 
term_dream = inverted_index[inverted_index["Term"]== "dream"]
movie_id_dream= term_dream["Movie_IDs"].iloc[0]

def intersection(list1, list2):
    if len(list1) < len(list2):
        smaller_list = list1
        bigger_list = list2
    else:
        smaller_list = list2
        bigger_list = list1
    intersected = []
    for item in smaller_list:
        for other_item in bigger_list:
            if item == other_item:
                intersected.append(item)
                break
    return intersected

movie_id_american_dream = intersection(movie_id_dream,movie_id_american)

#Print postinglist for Movies with the plotterms "dream" and "american" 
print(f"Posting list dream & american : {movie_id_american_dream}")
# Query for all films with american in plotterms and sort it by best rating
query_american = imdb[imdb["id"].isin(movie_id_american_dream)].sort_values(by="rating",ascending=False)
query_american.head(5) # Show only top 5 results


Posting list dream & american : [120669, 1343092, 2069861, 2267368, 13560124, 5092484, 2365001, 373862]


,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,['Dave has the American Dream and hates it. Hi...,dave american dream hates eccentric friends ha...
0,120669,movie,Fear and Loathing in Las Vegas,['English'],"['Adventure', 'Comedy', 'Drama']",7.5,Terry Gilliam,0,0,0,['An oddball journalist and his psychopathic l...,oddball journalist psychopathic lawyer travel...
1,1343092,movie,The Great Gatsby,['English'],"['Drama', 'Romance']",7.2,Baz Luhrmann,0,0,0,"['A writer and wall street trader, Nick, finds...",writer wall street trader nick finds drawn li...
3,2267368,movie,Joshua Tree,['English'],['Drama'],6.6,Li Cheng,0,0,0,['Set in the U.S. heartland following the 2008...,set heartland 2008 economic collapse divorced ...
6,2365001,movie,Immigrant,['English'],['Drama'],5.8,Barry Shurchin,0,0,0,['The coming of age story of a young Russian i...,coming age story young russian immigrant sear...


##### __(e) Extend your index to be able to rank the resulting items from Task 2d by the occurrence of search terms in the plotterms. Execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) again and print the resulting items.  [1 point]__

In [13]:
"""
Generating a dataframe with all plotterms and movie IDs associated with.
"""


#Creating Inverted Index Dataframe
inverted_index = pd.DataFrame({
    "Term": [],
    "Movie_IDs": []
})

for index, row in imdb.iterrows(): # Itterate through each row of the dataframe 
    movie_id = row["id"] # Get Movie ID from Every Movie
    for term in imdb.loc[index]["plotterms"].split(): # Itterate through every word in plotterms of said row
        if (inverted_index["Term"] != term).all(): # If plot word not already indexed
            occurrence = imdb.loc[index]["plotterms"].count(term)
            new_row = pd.DataFrame({"Term": [term], "Movie_IDs": [[[movie_id,occurrence]]]}) # generate new row for inverted index
            inverted_index = pd.concat([inverted_index, new_row], ignore_index=True) # add plotterm and movieid to inverted index
        elif not any(movie_id in x for x in inverted_index.loc[inverted_index["Term"] == term, "Movie_IDs"].iloc[0]): # search nested list for potentially already existing movieid
            occurrence = imdb.loc[index]["plotterms"].count(term)
            inverted_index.loc[inverted_index["Term"] == term, "Movie_IDs"].iloc[0].append([movie_id,occurrence]) # add movieid to plotterm index



inverted_index


,Term,Movie_IDs
0,oddball,"[[120669, 1]]"
1,journalist,"[[120669, 2], [12392504, 1]]"
2,psychopathic,"[[120669, 1], [477348, 1]]"
3,lawyer,"[[120669, 3], [2802850, 1], [3032476, 6]]"
4,travel,"[[120669, 1], [816692, 1], [6966692, 2], [2665..."
...,...,...
5552,principles,"[[200276, 1]]"
5553,tactical,"[[200276, 1]]"
5554,insights,"[[200276, 1]]"
5555,aspects,"[[200276, 1]]"


In [14]:
"""
Creation of the Posting df for the terms american
"""
# get Movie_IDs for the word american
term_american = inverted_index[inverted_index["Term"]== "american"]
movie_id_american = term_american["Movie_IDs"].iloc[0]

df_american = pd.DataFrame({
    "Movie_IDs": [],
    "Count": []
})
for element in movie_id_american:
    new_row = pd.DataFrame({"Movie_IDs": [element[0]], "Count": [element[1]]}) # generate new row for inverted index
    df_american = pd.concat([df_american, new_row], ignore_index=True) # add plotterm and movieid to inverted index

df_american



,Movie_IDs,Count
0,120669.0,5.0
1,1343092.0,3.0
2,2069861.0,3.0
3,2267368.0,2.0
4,13560124.0,2.0
5,5092484.0,2.0
6,2365001.0,2.0
7,373862.0,2.0
8,88763.0,1.0
9,46912.0,1.0


In [15]:
"""
Creation of the Posting df for the terms dream
"""
# get Movie_IDs for the word american
term_dream = inverted_index[inverted_index["Term"]== "dream"]
movie_id_dream = term_dream["Movie_IDs"].iloc[0]

df_dream = pd.DataFrame({
    "Movie_IDs": [],
    "Count": []
})
for element in movie_id_dream:
    new_row = pd.DataFrame({"Movie_IDs": [element[0]], "Count": [element[1]]}) # generate new row for inverted index
    df_dream = pd.concat([df_dream, new_row], ignore_index=True) # add plotterm and movieid to inverted index

df_dream

,Movie_IDs,Count
0,120669.0,4.0
1,1343092.0,4.0
2,2069861.0,3.0
3,2267368.0,2.0
4,13560124.0,2.0
5,5092484.0,2.0
6,2365001.0,2.0
7,373862.0,2.0
8,1375666.0,1.0
9,108778.0,3.0


In [16]:
"""
Merging the dream and american posting df and sorting it by occurence 
"""
df_dream_american = pd.merge(df_dream,df_american, how="inner", on="Movie_IDs")
df_dream_american.sort_values(by="Count_x" and "Count_y", ascending=False, inplace=True)
df_dream_american = df_dream_american.rename(columns={"Count_x": "Count dream", "Count_y": "Count american"})
df_dream_american['Total Count'] = df_dream_american['Count dream'] + df_dream_american['Count american']
df_dream_american

,Movie_IDs,Count dream,Count american,Total Count
0,120669.0,4.0,5.0,9.0
1,1343092.0,4.0,3.0,7.0
2,2069861.0,3.0,3.0,6.0
3,2267368.0,2.0,2.0,4.0
4,13560124.0,2.0,2.0,4.0
5,5092484.0,2.0,2.0,4.0
6,2365001.0,2.0,2.0,4.0
7,373862.0,2.0,2.0,4.0


In [17]:
"""
Creating a Posting list and query the query
"""
# Creating a cummultative score 
top_five_dream_american = df_dream_american.head(5)
postinglist = list(top_five_dream_american["Movie_IDs"])
imdb[imdb["id"].isin(postinglist)]

,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms
0,120669,movie,Fear and Loathing in Las Vegas,['English'],"['Adventure', 'Comedy', 'Drama']",7.5,Terry Gilliam,0,0,0,['An oddball journalist and his psychopathic l...,oddball journalist psychopathic lawyer travel...
1,1343092,movie,The Great Gatsby,['English'],"['Drama', 'Romance']",7.2,Baz Luhrmann,0,0,0,"['A writer and wall street trader, Nick, finds...",writer wall street trader nick finds drawn li...
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,['Dave has the American Dream and hates it. Hi...,dave american dream hates eccentric friends ha...
3,2267368,movie,Joshua Tree,['English'],['Drama'],6.6,Li Cheng,0,0,0,['Set in the U.S. heartland following the 2008...,set heartland 2008 economic collapse divorced ...
4,13560124,movie,After America,"['English', 'Somali']","['Drama', 'Fantasy', 'Mystery']",4.5,Jake Yuzna,0,0,0,['A group of criminal justice de-escalation wo...,group criminal justice deescalation workers m...


##### __(f) Use the TF-IDF score and the cosine similarity to execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) against all plots in the collection. Print the results with the corresponding ranking score. Also execute the query 'american' or 'dream' (the term 'american' or the term 'dream' should be contained in the plot).  [1.5 points]__

In [26]:
"""
TF-IDF
"""
import math
# Function used for counting words (plotterms, without stop words)
def count_words(text):
    return len(text.split())
# Calculate idf
count_of_documents = len(imdb)
count_of_documents_with_term_dream = len(df_dream)
count_of_documents_with_term_american = len(df_american)
idf_dream = math.log(count_of_documents/ count_of_documents_with_term_dream)
idf_american = math.log(count_of_documents/ count_of_documents_with_term_american)
# Calculate tf and tf idf
df_dream_american['tfidf_dream'] = (df_dream_american['Count dream']) / (imdb['plotterms'].apply(count_words)) * idf_dream
df_dream_american['tfidf_american'] = (df_dream_american['Count american']) / (imdb['plotterms'].apply(count_words)) * idf_american
df_dream_american['tfidf_Total'] = df_dream_american['tfidf_dream'] + df_dream_american['tfidf_american']
df_output = pd.merge(imdb, df_dream_american, left_on='id', right_on='Movie_IDs', how='inner')
df_output.sort_values(by='tfidf_Total', ascending=False)

,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms,Movie_IDs,Count dream,Count american,Total Count,tfidf_dream,tfidf_american,tfidf_Total
0,120669,movie,Fear and Loathing in Las Vegas,['English'],"['Adventure', 'Comedy', 'Drama']",7.5,Terry Gilliam,0,0,0,['An oddball journalist and his psychopathic l...,oddball journalist psychopathic lawyer travel...,120669.0,4.0,5.0,9.0,0.105249,0.112614,0.217863
5,5092484,movie,Beneath Us,"['English', 'Spanish']","['Horror', 'Thriller']",5.3,Max Pachman,0,0,0,['The American Dream becomes a nightmare for a...,american dream nightmare group undocumented d...,5092484.0,2.0,2.0,4.0,0.117309,0.100414,0.217723
3,2267368,movie,Joshua Tree,['English'],['Drama'],6.6,Li Cheng,0,0,0,['Set in the U.S. heartland following the 2008...,set heartland 2008 economic collapse divorced ...,2267368.0,2.0,2.0,4.0,0.114915,0.098365,0.213280
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,['Dave has the American Dream and hates it. Hi...,dave american dream hates eccentric friends ha...,2069861.0,3.0,3.0,6.0,0.112616,0.096398,0.209014
4,13560124,movie,After America,"['English', 'Somali']","['Drama', 'Fantasy', 'Mystery']",4.5,Jake Yuzna,0,0,0,['A group of criminal justice de-escalation wo...,group criminal justice deescalation workers m...,13560124.0,2.0,2.0,4.0,0.106242,0.090941,0.197183
7,373862,movie,Getno,"['Hungarian', 'English']",['Comedy'],4.0,András Salamon,0,0,0,"['Following the promise of the American Dream,...",promise american dream jancsi major family wh...,373862.0,2.0,2.0,4.0,0.092308,0.079015,0.171323
6,2365001,movie,Immigrant,['English'],['Drama'],5.8,Barry Shurchin,0,0,0,['The coming of age story of a young Russian i...,coming age story young russian immigrant sear...,2365001.0,2.0,2.0,4.0,0.061205,0.052390,0.113595
1,1343092,movie,The Great Gatsby,['English'],"['Drama', 'Romance']",7.2,Baz Luhrmann,0,0,0,"['A writer and wall street trader, Nick, finds...",writer wall street trader nick finds drawn li...,1343092.0,4.0,3.0,7.0,0.036095,0.023173,0.059268


In [19]:
"""
Cosine Similarity :
creating vectors for cosine similarity comparison 
"""

#Creating Inverted Index Dataframe
plot_vectors = pd.DataFrame({
    "Movie_ID": [],
    "Terms": []
})

for index, row in imdb.iterrows(): # Itterate through each row of the dataframe 
    movie_id = row["id"] # Get Movie ID from Every Movie
    vector = []
    for term in inverted_index["Term"]: # Itterate through every unique plotterm
        occurrence = imdb.loc[index]["plotterms"].count(term)
        vector.append(occurrence)
    new_row = pd.DataFrame({"Movie_ID": [movie_id], "Terms": [vector]}) # generate new row for inverted index
    plot_vectors = pd.concat([plot_vectors, new_row], ignore_index=True) # add plotterm and movieid to inverted index


plot_vectors

,Movie_ID,Terms
0,120669.0,"[1, 2, 1, 3, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, ..."
1,1343092.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2069861.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2267368.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,13560124.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
162,2085059.0,"[0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, ..."
163,2707408.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
164,98936.0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
165,4934214.0,"[0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
"""
Cosine Similarity :
calculation and output
"""
from scipy import spatial
american_dream_vectors = plot_vectors[plot_vectors["Movie_ID"].isin(df_dream_american["Movie_IDs"])]
non_american_dream_vectors = plot_vectors[~plot_vectors["Movie_ID"].isin(df_dream_american["Movie_IDs"])]

data = []  # Initialize an empty list to store the similarity data
for vector_american_dream in american_dream_vectors["Terms"]:
    inner_data = []  # Initialize an empty list to store inner similarity data

    # Iterate over the vectors in "Terms" from non_american_dream_vectors
    for vector_selection in non_american_dream_vectors["Terms"]:
        # Calculate the cosine similarity between the current pair of vectors
        sim = 1 - spatial.distance.cosine(vector_american_dream, vector_selection)
        inner_data.append(sim)  # Append the similarity score to the inner_data list

    data.append(inner_data)  # Append the inner_data list to the final data list

# Create a DataFrame from the similarity data
df_cosine_sim = pd.DataFrame(data, index=[f'American&Dream containing Vector ID: {i}' for i in american_dream_vectors["Movie_ID"]], columns=[f'Vector_ID: {i}' for i in non_american_dream_vectors["Movie_ID"]])

# Transpose the DataFrame
df_cosine_sim = df_cosine_sim.T

# Display the resulting DataFrame
df_cosine_sim

,American&Dream containing Vector ID: 120669.0,American&Dream containing Vector ID: 1343092.0,American&Dream containing Vector ID: 2069861.0,American&Dream containing Vector ID: 2267368.0,American&Dream containing Vector ID: 13560124.0,American&Dream containing Vector ID: 5092484.0,American&Dream containing Vector ID: 2365001.0,American&Dream containing Vector ID: 373862.0
Vector_ID: 111161.0,0.135698,0.153428,0.163556,0.076447,0.130536,0.139195,0.090738,0.109562
Vector_ID: 68646.0,0.216673,0.259119,0.196910,0.167161,0.167018,0.222746,0.262603,0.287486
Vector_ID: 468569.0,0.255284,0.299995,0.182612,0.150878,0.265861,0.233742,0.261246,0.310451
Vector_ID: 108052.0,0.134401,0.215816,0.164423,0.048795,0.098667,0.170506,0.205494,0.173081
Vector_ID: 110912.0,0.320574,0.408532,0.257112,0.243599,0.353925,0.339182,0.270604,0.306450
...,...,...,...,...,...,...,...,...
Vector_ID: 2085059.0,0.280883,0.417381,0.256889,0.276026,0.300778,0.277163,0.305576,0.382407
Vector_ID: 2707408.0,0.368689,0.339638,0.195691,0.263371,0.275676,0.224009,0.267343,0.261112
Vector_ID: 98936.0,0.328875,0.428386,0.258859,0.271967,0.371990,0.323974,0.352486,0.387875
Vector_ID: 4934214.0,0.267425,0.352013,0.241476,0.165188,0.321753,0.311968,0.235468,0.336273
